In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import matplotlib as mpl
import os
from matplotlib.lines import Line2D
from collections import Counter
import math
from sklearn.decomposition import PCA
from aging.behavior.syllables import relabel_by_usage
from tqdm import tqdm
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
import random
import scipy
from scipy import stats
from aging.plotting import format_plots, PlotConfig, save_factory, figure, legend, format_pizza_plots

In [2]:
format_plots()
#format_pizza_plots()

In [3]:
from matplotlib.colors import LinearSegmentedColormap
# Define the color map
colors = ['#dadaeb','#6a51a3']
custom_cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
cmm=custom_cmap

In [4]:
## update data
def filter_df(df):
    max=39
    thresh=8
    syll=df.columns.values[df.columns.values>max]
    df.drop(syll,axis=1, inplace=True)
    age_counts = df.index.get_level_values('age').value_counts()
    ages_greater = list(age_counts[age_counts > thresh].index)
    return df.loc[data.index.get_level_values('age').isin(ages_greater)]

In [5]:
path = Path('/n/groups/datta/win/longtogeny/data/ontogeny/version_11/longtogeny_males_relabeled_usage_matrix_v00.parquet')
df = pd.read_parquet(path)

In [6]:
# arrange data
data = df.astype(float, errors='ignore')
data= filter_df(data).groupby(['age','uuid','mouse']).mean()
data = data.query('age<100')
data_all = data[sorted(data.columns)].copy()

In [7]:
data_labels = data_all.index.get_level_values('age').to_numpy()
data = data_all.to_numpy()

In [8]:
temp = data_all.copy()
temp.reset_index(inplace=True)

if 'level_0' in temp.columns:
    temp.drop(columns=['level_0'], inplace=True)

In [9]:
# for all syllables
syll = data_all.columns
n=len(syll)
s=[]
p=[]
h=[]
a = 0.05

for i in syll:
    # Melt the DataFrame
    melted_df = pd.melt(temp, id_vars='age', value_vars=[i], value_name='value')
    melted_df.drop(columns=['variable'], inplace=True)
    h_statistic, p_value = stats.kruskal(*[group['value'] for name, group in melted_df.groupby('age')])

    # Print the results
    print("H-statistic:", h_statistic)
    print("p-value:", p_value)
    cp = p_value*n
    if cp<0.05:
        s.extend([i])
        p.extend([cp])
        h.extend([h])

H-statistic: 411.3332629590566
p-value: 1.3592853104749496e-52
H-statistic: 495.53567974739474
p-value: 2.0106983057612795e-68
H-statistic: 227.49904208579898
p-value: 1.8698331205480778e-20
H-statistic: 247.0330598587914
p-value: 1.2857485156138399e-23
H-statistic: 423.7742365368904
p-value: 6.672577504014801e-55
H-statistic: 337.48207171008704
p-value: 3.674888444099864e-39
H-statistic: 464.6091956405309
p-value: 1.4769764199435667e-62
H-statistic: 286.7438221047404
p-value: 2.7656475963556246e-30
H-statistic: 198.7481106293781
p-value: 5.599837881097711e-16
H-statistic: 464.76663943023425
p-value: 1.379275572053763e-62
H-statistic: 302.7241672546985
p-value: 4.8311806819529325e-33
H-statistic: 443.9103579447273
p-value: 1.1572657496207915e-58
H-statistic: 327.4900295405541
p-value: 2.186580613021329e-37
H-statistic: 145.04725006982602
p-value: 2.0440985109390195e-08
H-statistic: 264.95588467146956
p-value: 1.3669302671086055e-26
H-statistic: 391.3905852768039
p-value: 6.446595360839

In [12]:
print(len(s))

40


In [10]:
melted_df = pd.melt(temp, id_vars=['age','mouse'], value_vars=[0], value_name='value')
melted_df.drop(columns=['variable'], inplace=True)

In [11]:
from statsmodels.stats.anova import AnovaRM

#perform the repeated measures ANOVA
print(AnovaRM(data=melted_df, depvar='value', subject='mouse',within=['age']).fit())

ValueError: Data is unbalanced.